In [11]:
import pandas as pd
import duckdb

In [12]:
# insert data
sales_table_creation_query = """
drop table if exists sales;

create table sales (date date, product varchar, sales int);

insert into sales values
  ('2024-01-12', 'coffee', 789),
  ('2024-01-13', 'tea', 605),
  ('2024-01-10', 'coffee', 509),
  ('2024-01-14', 'tea', 340),
  ('2024-01-12', 'tea', 500),
  ('2024-01-11', 'coffee', 423),
  ;
SELECT * FROM sales
"""

In [13]:
# transform sales table to a pandas dataframe
df = duckdb.sql(sales_table_creation_query).df()
df

,date,product,sales
0,2024-01-12,coffee,789
1,2024-01-13,tea,605
2,2024-01-10,coffee,509
3,2024-01-14,tea,340
4,2024-01-12,tea,500
5,2024-01-11,coffee,423


# 1. Moving average

## 1a. SQL

In [6]:
sql_query = """ 
    SELECT 
        date,
        product,
        sales,
        AVG(sales) OVER (PARTITION BY product ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_avg
    FROM sales
    """

duckdb.sql(sql_query)

┌────────────┬─────────┬───────┬───────────────────┐
│    date    │ product │ sales │    moving_avg     │
│    date    │ varchar │ int32 │      double       │
├────────────┼─────────┼───────┼───────────────────┤
│ 2024-01-10 │ coffee  │   509 │             509.0 │
│ 2024-01-11 │ coffee  │   423 │             466.0 │
│ 2024-01-12 │ coffee  │   789 │ 573.6666666666666 │
│ 2024-01-12 │ tea     │   500 │             500.0 │
│ 2024-01-13 │ tea     │   605 │             552.5 │
│ 2024-01-14 │ tea     │   340 │ 481.6666666666667 │
└────────────┴─────────┴───────┴───────────────────┘

## 1b. Pandas

In [23]:
df1 = df.copy()

df1["moving_avg"] = df1.sort_values(['date'])\
                        .groupby('product')['sales']\
                        .transform(lambda x: x.rolling(3, min_periods=1).mean())
df1.sort_values(['date', 'product'])

,date,product,sales,moving_avg
2,2024-01-10,coffee,509,509.000000
5,2024-01-11,coffee,423,466.000000
0,2024-01-12,coffee,789,573.666667
4,2024-01-12,tea,500,500.000000
1,2024-01-13,tea,605,552.500000
3,2024-01-14,tea,340,481.666667


# 2. Running total

## 2a. SQL

In [17]:
# Running total
sql_query = """ 
    SELECT 
        date,
        product,
        sales,
        SUM(sales) OVER (PARTITION BY product ORDER BY date ROWS UNBOUNDED PRECEDING) AS running_total_sales
    FROM sales
    """

duckdb.sql(sql_query)

┌────────────┬─────────┬───────┬─────────────────────┐
│    date    │ product │ sales │ running_total_sales │
│    date    │ varchar │ int32 │       int128        │
├────────────┼─────────┼───────┼─────────────────────┤
│ 2024-01-10 │ coffee  │   509 │                 509 │
│ 2024-01-11 │ coffee  │   423 │                 932 │
│ 2024-01-12 │ coffee  │   789 │                1721 │
│ 2024-01-12 │ tea     │   500 │                 500 │
│ 2024-01-13 │ tea     │   605 │                1105 │
│ 2024-01-14 │ tea     │   340 │                1445 │
└────────────┴─────────┴───────┴─────────────────────┘

## 2b. Pandas

In [22]:
df2 = df.copy()

# Running total 
df2["running_total_sales"] = df2.sort_values(['date'])\
                                        .groupby('product')['sales']\
                                        .cumsum()
df2.sort_values(['date', 'product'])

,date,product,sales,running_total_sales
2,2024-01-10,coffee,509,509
5,2024-01-11,coffee,423,932
0,2024-01-12,coffee,789,1721
4,2024-01-12,tea,500,500
1,2024-01-13,tea,605,1105
3,2024-01-14,tea,340,1445


# 3. LAG/ LEAD

## 3a. SQL

In [25]:
# Previous day sales and following day sales
sql_query = """ 
    SELECT 
        date,
        product,
        sales,
        LAG(sales) OVER (PARTITION BY product ORDER BY date) AS previous_day_sales,
        LEAD(sales) OVER (PARTITION BY product ORDER BY date) AS following_day_sales
    FROM sales
    """

duckdb.sql(sql_query)

┌────────────┬─────────┬───────┬────────────────────┬─────────────────────┐
│    date    │ product │ sales │ previous_day_sales │ following_day_sales │
│    date    │ varchar │ int32 │       int32        │        int32        │
├────────────┼─────────┼───────┼────────────────────┼─────────────────────┤
│ 2024-01-10 │ coffee  │   509 │               NULL │                 423 │
│ 2024-01-11 │ coffee  │   423 │                509 │                 789 │
│ 2024-01-12 │ coffee  │   789 │                423 │                NULL │
│ 2024-01-12 │ tea     │   500 │               NULL │                 605 │
│ 2024-01-13 │ tea     │   605 │                500 │                 340 │
│ 2024-01-14 │ tea     │   340 │                605 │                NULL │
└────────────┴─────────┴───────┴────────────────────┴─────────────────────┘

## 3b. Pandas

In [26]:
df3 = df.copy()

# Previous day sales and following day sales
df3["previous_day_sales"] = df3.sort_values(['date'])\
                                        .groupby('product')['sales']\
                                        .shift(1)
df3["following_day_sales"] = df3.sort_values(['date'])\
                                        .groupby('product')['sales']\
                                        .shift(-1)
df3.sort_values(['date', 'product'])

,date,product,sales,previous_day_sales,following_day_sales
2,2024-01-10,coffee,509,NaN,423.0
5,2024-01-11,coffee,423,509.0,789.0
0,2024-01-12,coffee,789,423.0,NaN
4,2024-01-12,tea,500,NaN,605.0
1,2024-01-13,tea,605,500.0,340.0
3,2024-01-14,tea,340,605.0,NaN


# 4. RANK()

## 4a. SQL

In [62]:
sql_query = """ 
    SELECT 
        date,
        product,
        sales,
        RANK() OVER (PARTITION BY product ORDER BY sales DESC) AS rank_nb
    FROM sales
    """

duckdb.sql(sql_query)

┌────────────┬─────────┬───────┬─────────┐
│    date    │ product │ sales │ rank_nb │
│    date    │ varchar │ int32 │  int64  │
├────────────┼─────────┼───────┼─────────┤
│ 2024-01-12 │ coffee  │   789 │       1 │
│ 2024-01-10 │ coffee  │   509 │       2 │
│ 2024-01-11 │ coffee  │   423 │       3 │
│ 2024-01-13 │ tea     │   605 │       1 │
│ 2024-01-12 │ tea     │   500 │       2 │
│ 2024-01-14 │ tea     │   340 │       3 │
└────────────┴─────────┴───────┴─────────┘

## 4b. Pandas

In [69]:
df4 = df.copy()

df4['rank_nb'] = df4.sort_values(['date'])\
                 .groupby('product')['sales']\
                 .rank(ascending=False)

df4.sort_values(['product', 'sales'], ascending=[True, False])

,date,product,sales,rank_nb
0,2024-01-12,coffee,789,1.0
2,2024-01-10,coffee,509,2.0
5,2024-01-11,coffee,423,3.0
1,2024-01-13,tea,605,1.0
4,2024-01-12,tea,500,2.0
3,2024-01-14,tea,340,3.0


# Summary

<img src="assets/window_functions_with_partition_1.png" width=1000 />
<img src="assets/window_functions_with_partition_2.png" width=1000 />